# This notebook mimics "D-ID" image to Animated Head model used by MyHeritage Deep Nostalgia: https://www.myheritage.com/deep-nostalgia?ref=louisbouchard.ai

# The model functions by adapting the First Order For Image Animation: https://github.com/AliaksandrSiarohin/first-order-model

### Author: Dusan Birtasevic
### Date: July.31.2023

In [1]:
# Clone the repository
!git clone https://github.com/AliaksandrSiarohin/first-order-model

# Navigate to the directory
%cd first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 393, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 393 (delta 41), reused 64 (delta 38), pack-reused 312
Receiving objects: 100% (393/393), 72.19 MiB | 17.10 MiB/s, done.
Resolving deltas: 100% (204/204), done.
Updating files: 100% (48/48), done.
/content/first-order-model


In [2]:
# Download the VoxCeleb model which is trained for facial animations
!pip install ffmpeg-python
!apt install ffmpeg
!pip install gdown
!gdown --id 1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ
#!wget "https://disk.yandex.com/d/lEw8uRm140L_eQ" -O vox-cpk.pth.tar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1L8P-hpBhZi8Q_1vP2KlQ4N6dvlzpYBvZ
To: /content/first-order-model/vox-adv-cpk.pth.tar
100% 751M/751M [00:09<00:00, 81.8MB/s]


In [3]:
from google.colab import files
import imageio

# Upload source image
uploaded = files.upload()
source_image_path = list(uploaded.keys())[0]
source_image = imageio.imread(source_image_path)

# Upload driving video
uploaded = files.upload()
driving_video_path = list(uploaded.keys())[0]
reader = imageio.get_reader(driving_video_path)
driving_video = [frame for frame in reader]

Saving neymar.jpg to neymar.jpg


<ipython-input-3-4afefb3abbfc>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  source_image = imageio.imread(source_image_path)


Saving driving.mp4 to driving.mp4


In [4]:
#import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import Video, display
import warnings
warnings.filterwarnings("ignore")

# Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml',
                            checkpoint_path='vox-adv-cpk.pth.tar')

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

  0%|          | 0/348 [00:00<?, ?it/s]

In [5]:
# Save directly using imageio
animation_path = source_image_path.rsplit('.', 1)[0] + "_output.mp4"
imageio.mimsave(animation_path, [img_as_ubyte(frame) for frame in predictions], fps=20)

import os
if os.path.exists(animation_path):
    print(f"File {animation_path} saved successfully!")
else:
    print(f"Failed to save {animation_path}.")

File neymar_output.mp4 saved successfully!


In [7]:
from IPython.display import HTML
from base64 import b64encode

def play_video(video_path):
    # Load the video
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # Display the video
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

video_path = f"/content/first-order-model/{animation_path}"
display(play_video(video_path))